In [1]:
import time, os, math, numpy as np, pandas as pd, sklearn as sk, matplotlib as plt
# import sqlite3
from pandasql import sqldf
import seaborn as sns
pysqldf = lambda q: sqldf(q, globals())
np.random.seed(42)
from pandas import get_dummies
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold,cross_val_score, cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc,classification_report, make_scorer, recall_score, mean_squared_error,precision_recall_curve, precision_score, auc
from sklearn.metrics import roc_auc_score as ras
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,ExtraTreesClassifier, VotingClassifier, StackingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler, MinMaxScaler, normalize
from imblearn.over_sampling import SMOTE, RandomOverSampler
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('c:/Users/snooney/Desktop/DOJ Hackathon/final_data_mean.csv')

In [7]:
df_remove = ['Estimate_Households_Total',
'Estimate_Households_Total_Less than $10,000',
 'Estimate_Households_Total_$10,000 to $14,999',
 'Estimate_Households_Total_$15,000 to $24,999',
 'Estimate_Households_Total_$25,000 to $34,999',
 'Estimate_Households_Total_$35,000 to $49,999',
 'Estimate_Households_Total_$50,000 to $74,999',
 'Estimate_Households_Total_$75,000 to $99,999',
 'Estimate_Households_Total_$100,000 to $149,999',
 'Estimate_Households_Total_$150,000 to $199,999',
 'Estimate_Households_Total_$200,000 or more',
 'Total population_Male',
 'Total population_Female',
 'males per 100 females',
 'Percent_Total population_18 years and over',
 'Percent_Total population_18 years and over_Male',
 'Percent_Total population_18 years and over_Female',
 'Percent_Total population_65 years and over_Male',
 'Percent_Total population_65 years and over_Female',
 'Percent_RACE_Total population_One race_American Indian and Alaska Native',
 'Percent_RACE_Total population_One race_American Indian and Alaska Native_Cherokee tribal grouping',
 'Percent_RACE_Total population_One race_American Indian and Alaska Native_Chippewa tribal grouping',
 'Percent_RACE_Total population_One race_American Indian and Alaska Native_Navajo tribal grouping',
 'Percent_RACE_Total population_One race_American Indian and Alaska Native_Sioux tribal grouping',
 'Percent_RACE_Total population_One race_Asian',
 'Percent_RACE_Total population_One race_Asian_Asian Indian',
 'Percent_RACE_Total population_One race_Asian_Chinese',
 'Percent_RACE_Total population_One race_Asian_Filipino',
 'Percent_RACE_Total population_One race_Asian_Japanese',
 'Percent_RACE_Total population_One race_Asian_Korean',
 'Percent_RACE_Total population_One race_Asian_Vietnamese',
 'Percent_RACE_Total population_One race_Asian_Other Asian',
 'Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander',
 'Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Native Hawaiian',
 'Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Chamorro',
 'Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Samoan',
 'Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Other Pacific Islander',
 'Percent_RACE_Total population_One race_Some other race',
 'Percent_HISPANIC OR LATINO AND RACE_Total population_Hispanic or Latino (of any race)',
 'Unnamed: 0',
 'HPSA Score']

In [8]:

df['Percent_RACE_Total population_One race_American Indian and Alaska Native Total'] = df[['Percent_RACE_Total population_One race_American Indian and Alaska Native_Cherokee tribal grouping', 'Percent_RACE_Total population_One race_American Indian and Alaska Native_Chippewa tribal grouping', 'Percent_RACE_Total population_One race_American Indian and Alaska Native_Navajo tribal grouping', 'Percent_RACE_Total population_One race_American Indian and Alaska Native_Sioux tribal grouping']].sum(axis=1)
df['Percent_RACE_Total population_One race_Asian_Total'] = df[['Percent_RACE_Total population_One race_Asian_Asian Indian','Percent_RACE_Total population_One race_Asian_Chinese','Percent_RACE_Total population_One race_Asian_Filipino','Percent_RACE_Total population_One race_Asian_Japanese','Percent_RACE_Total population_One race_Asian_Korean','Percent_RACE_Total population_One race_Asian_Vietnamese','Percent_RACE_Total population_One race_Asian_Other Asian']].sum(axis=1)
df['Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Total'] = df[['Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Native Hawaiian','Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Chamorro','Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Samoan','Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Other Pacific Islander']].sum(axis=1)
df['Percent_RACE_Total population_Other_Hispanic'] = df[['Percent_RACE_Total population_One race_Some other race', 'Percent_HISPANIC OR LATINO AND RACE_Total population_Hispanic or Latino (of any race)']].sum(axis=1)

In [9]:
df.drop(columns=df_remove, inplace=True)

Feature Importance

In [10]:
# Fill missing values in each column with respective mode value
for column in df.columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

# Assuming you have X as features and y as labels
X = df.drop('HPSA Category', axis=1)  # all features excluding 'HPSA Category'
y = df['HPSA Category']  # target variable

# Create a Logistic Regression classifier
lr = LogisticRegression(max_iter=10000)

# Create RFECV object with custom step function for progress bar
rfecv = RFECV(estimator=lr, step=1, cv=StratifiedKFold(10), scoring='f1')

# Fit RFECV
rfecv.fit(X, y)

# Print the optimal number of features
print('Optimal number of features: {}'.format(rfecv.n_features_))

# Get the features sorted by importance
feature_importance = list(zip(X.columns, rfecv.estimator_.coef_[0]))
feature_importance = sorted(feature_importance, key=lambda x:x[1], reverse=True)

# Print the most important features
for feature, importance in feature_importance:
    print('Feature: {}, Importance: {}'.format(feature, importance))

# Keep only the most important features
X_important = rfecv.transform(X)


Optimal number of features: 3
Feature: ACCESS2, Importance: 2.3841686285341184
Feature: ARTHRITIS, Importance: -0.07636894239880031
Feature: CountyFIPS, Importance: -2.2786476314987243


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

# Assuming X as features and y as labels
X = df.drop('HPSA Category', axis=1)  # all features excluding 'HPSA Category'
y = df['HPSA Category']  # target variable

# Standardize the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

# Create a train-test split
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2, random_state=42)

# Create a Logistic Regression classifier with L1 regularization
lr = LogisticRegression(penalty='l1', solver='liblinear', C=0.1, max_iter=10000)

# Fit the model
lr.fit(X_train, y_train)

# Get the coefficients
coef = lr.coef_[0]

# Create a mask for non-zero coefficients
important_features_mask = coef != 0

# Apply the mask to get the names of important features
important_features = X.columns[important_features_mask]

# Print the important features
print('Important features:', important_features)

# Keep only the important features
X_important_train = X_train[:, important_features_mask]
X_important_test = X_test[:, important_features_mask]

# Fit the model with important features
lr.fit(X_important_train, y_train)

# Predict and compute F1 score on the training data
y_train_pred = lr.predict(X_important_train)
f1_train = f1_score(y_train, y_train_pred)
print('Training F1 Score: ', f1_train)

# Predict and compute F1 score on the testing data
y_test_pred = lr.predict(X_important_test)
f1_test = f1_score(y_test, y_test_pred)
print('Testing F1 Score: ', f1_test)


Important features: Index(['CountyFIPS', 'ACCESS2', 'ARTHRITIS', 'BINGE', 'BPHIGH', 'CANCER',
       'CHECKUP', 'CHOLSCREEN', 'COPD', 'COREM', 'COREW', 'DENTAL',
       'DEPRESSION', 'DISABILITY', 'GHLTH', 'HEARING', 'LPA', 'MAMMOUSE',
       'MOBILITY', 'OBESITY', 'SLEEP', 'STROKE',
       'Estimate_Households_Median income (dollars)',
       'Estimate_Total housing units',
       'Percent_Total population_35 to 64 years',
       'Percent_Total population_65 years and over',
       'Percent_RACE_Total population_Two or more races',
       'Percent_RACE_Total population_One race_White', 'DrugOrAlc_Deaths_pct',
       'Percent_RACE_Total population_One race_Asian_Total',
       'Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Total'],
      dtype='object')
Training F1 Score:  0.7376048817696415
Testing F1 Score:  0.7151515151515151
